# AWS Full Stack с Kubernetes (EKS)

## Развертывание микросервисного приложения на EKS

Этот туториал демонстрирует развертывание полноценного микросервисного приложения на Amazon EKS с использованием:

- **EKS**: Managed Kubernetes кластер
- **Helm**: Управление Kubernetes приложениями
- **ArgoCD**: GitOps для CD
- **Prometheus + Grafana**: Мониторинг
- **Istio**: Service Mesh

### Архитектура

```
┌─────────────────────────────────────────┐
│  Route53 (DNS)                          │
└──────────────┬──────────────────────────┘
               │
┌──────────────▼──────────────────────────┐
│  ALB Ingress Controller                 │
└──────────────┬──────────────────────────┘
               │
┌──────────────▼──────────────────────────┐
│  Istio Gateway                          │
└──────────────┬──────────────────────────┘
               │
    ┌──────────┴──────────┐
    │                     │
┌───▼────┐  ┌──────▼──────┐  ┌────────┐
│Frontend│  │   API GW    │  │ Auth   │
│Service │  │   Service   │  │Service │
└────────┘  └─────────────┘  └────────┘
                  │
         ┌────────┴────────┐
    ┌────▼─────┐   ┌──────▼──────┐
    │ Orders   │   │  Products   │
    │ Service  │   │  Service    │
    └──────────┘   └─────────────┘
         │                │
    ┌────▼─────┐   ┌─────▼──────┐
    │PostgreSQL│   │   Redis    │
    │  (RDS)   │   │(ElastiCache│
    └──────────┘   └────────────┘
```

## Шаг 1: Создание EKS кластера

In [ ]:
%%writefile eks-cluster.tf

module "eks" {
  source = "git::https://github.com/v-grand/infra-k8s.git//modules/eks"

  cluster_name    = "my-app-cluster"
  cluster_version = "1.28"

  vpc_id     = module.network.vpc_id
  subnet_ids = module.network.private_subnets

  # Node Groups
  node_groups = {
    general = {
      desired_capacity = 3
      max_capacity     = 10
      min_capacity     = 2

      instance_types = ["t3.medium"]
      capacity_type  = "ON_DEMAND"
      
      labels = {
        role = "general"
      }
    }
    
    spot = {
      desired_capacity = 2
      max_capacity     = 5
      min_capacity     = 0

      instance_types = ["t3.medium", "t3a.medium"]
      capacity_type  = "SPOT"
      
      labels = {
        role = "spot"
      }
      
      taints = [{
        key    = "spot"
        value  = "true"
        effect = "NoSchedule"
      }]
    }
  }

  # Enable IRSA (IAM Roles for Service Accounts)
  enable_irsa = true

  # Cluster addons
  cluster_addons = {
    coredns = {
      most_recent = true
    }
    kube-proxy = {
      most_recent = true
    }
    vpc-cni = {
      most_recent = true
    }
  }
}

# Update kubeconfig
resource "null_resource" "update_kubeconfig" {
  depends_on = [module.eks]
  
  provisioner "local-exec" {
    command = "aws eks update-kubeconfig --name ${module.eks.cluster_name} --region us-east-1"
  }
}

In [ ]:
# Развертывание EKS кластера
!terraform init
!terraform apply -auto-approve

# Проверка кластера
!kubectl get nodes
!kubectl get pods -A

## Шаг 2: Установка базовых компонентов

### ALB Ingress Controller

In [ ]:
# Установка AWS Load Balancer Controller
!helm repo add eks https://aws.github.io/eks-charts
!helm repo update

!helm install aws-load-balancer-controller eks/aws-load-balancer-controller \
  -n kube-system \
  --set clusterName=my-app-cluster \
  --set serviceAccount.create=true \
  --set serviceAccount.name=aws-load-balancer-controller

# Проверка
!kubectl get deployment -n kube-system aws-load-balancer-controller

### Istio Service Mesh

In [ ]:
# Установка Istio
!curl -L https://istio.io/downloadIstio | sh -
!cd istio-* && export PATH=$PWD/bin:$PATH

# Установка с профилем demo
!istioctl install --set profile=demo -y

# Включение автоматического injection sidecar
!kubectl label namespace default istio-injection=enabled

# Проверка
!kubectl get pods -n istio-system

## Шаг 3: Развертывание приложения

### Создание namespace и secrets

In [ ]:
%%writefile k8s/namespace.yaml

apiVersion: v1
kind: Namespace
metadata:
  name: my-app
  labels:
    istio-injection: enabled
---
apiVersion: v1
kind: Secret
metadata:
  name: app-secrets
  namespace: my-app
type: Opaque
stringData:
  DB_HOST: "my-app-db.xxxxx.us-east-1.rds.amazonaws.com"
  DB_NAME: "myapp"
  DB_USER: "admin"
  DB_PASSWORD: "ENCRYPTED_WITH_SOPS"
  REDIS_URL: "redis://my-app-redis.xxxxx.cache.amazonaws.com:6379"
  JWT_SECRET: "ENCRYPTED_WITH_SOPS"

In [ ]:
# Расшифровка и применение секретов
!sops -d k8s/namespace.yaml | kubectl apply -f -

### Развертывание микросервисов

In [ ]:
%%writefile k8s/deployments.yaml

# Frontend Service
apiVersion: apps/v1
kind: Deployment
metadata:
  name: frontend
  namespace: my-app
spec:
  replicas: 3
  selector:
    matchLabels:
      app: frontend
  template:
    metadata:
      labels:
        app: frontend
        version: v1
    spec:
      containers:
      - name: frontend
        image: my-ecr-repo/frontend:latest
        ports:
        - containerPort: 80
        resources:
          requests:
            memory: "128Mi"
            cpu: "100m"
          limits:
            memory: "256Mi"
            cpu: "200m"
---
apiVersion: v1
kind: Service
metadata:
  name: frontend
  namespace: my-app
spec:
  selector:
    app: frontend
  ports:
  - port: 80
    targetPort: 80
---
# API Gateway Service
apiVersion: apps/v1
kind: Deployment
metadata:
  name: api-gateway
  namespace: my-app
spec:
  replicas: 3
  selector:
    matchLabels:
      app: api-gateway
  template:
    metadata:
      labels:
        app: api-gateway
        version: v1
    spec:
      containers:
      - name: api-gateway
        image: my-ecr-repo/api-gateway:latest
        ports:
        - containerPort: 8000
        envFrom:
        - secretRef:
            name: app-secrets
        resources:
          requests:
            memory: "256Mi"
            cpu: "200m"
          limits:
            memory: "512Mi"
            cpu: "500m"
        livenessProbe:
          httpGet:
            path: /health
            port: 8000
          initialDelaySeconds: 30
          periodSeconds: 10
        readinessProbe:
          httpGet:
            path: /ready
            port: 8000
          initialDelaySeconds: 10
          periodSeconds: 5
---
apiVersion: v1
kind: Service
metadata:
  name: api-gateway
  namespace: my-app
spec:
  selector:
    app: api-gateway
  ports:
  - port: 8000
    targetPort: 8000

In [ ]:
!kubectl apply -f k8s/deployments.yaml
!kubectl get pods -n my-app -w

### Istio Gateway и VirtualService

In [ ]:
%%writefile k8s/istio-gateway.yaml

apiVersion: networking.istio.io/v1beta1
kind: Gateway
metadata:
  name: my-app-gateway
  namespace: my-app
spec:
  selector:
    istio: ingressgateway
  servers:
  - port:
      number: 80
      name: http
      protocol: HTTP
    hosts:
    - "myapp.example.com"
  - port:
      number: 443
      name: https
      protocol: HTTPS
    tls:
      mode: SIMPLE
      credentialName: myapp-tls-cert
    hosts:
    - "myapp.example.com"
---
apiVersion: networking.istio.io/v1beta1
kind: VirtualService
metadata:
  name: my-app
  namespace: my-app
spec:
  hosts:
  - "myapp.example.com"
  gateways:
  - my-app-gateway
  http:
  - match:
    - uri:
        prefix: "/api"
    route:
    - destination:
        host: api-gateway
        port:
          number: 8000
  - match:
    - uri:
        prefix: "/"
    route:
    - destination:
        host: frontend
        port:
          number: 80

In [ ]:
!kubectl apply -f k8s/istio-gateway.yaml

## Шаг 4: ArgoCD для GitOps

In [ ]:
# Установка ArgoCD
!kubectl create namespace argocd
!kubectl apply -n argocd -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml

# Получение пароля admin
!kubectl -n argocd get secret argocd-initial-admin-secret -o jsonpath="{.data.password}" | base64 -d

# Port forward для доступа к UI
!kubectl port-forward svc/argocd-server -n argocd 8080:443

## Шаг 5: Мониторинг (Prometheus + Grafana)

In [ ]:
# Установка Prometheus Stack через Helm
!helm repo add prometheus-community https://prometheus-community.github.io/helm-charts
!helm repo update

!helm install prometheus prometheus-community/kube-prometheus-stack \
  --namespace monitoring \
  --create-namespace \
  --set prometheus.prometheusSpec.serviceMonitorSelectorNilUsesHelmValues=false \
  --set grafana.adminPassword=admin123

# Проверка
!kubectl get pods -n monitoring

## Шаг 6: Автоскейлинг (HPA + Cluster Autoscaler)

In [ ]:
%%writefile k8s/hpa.yaml

apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: api-gateway-hpa
  namespace: my-app
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: api-gateway
  minReplicas: 3
  maxReplicas: 10
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 70
  - type: Resource
    resource:
      name: memory
      target:
        type: Utilization
        averageUtilization: 80

In [ ]:
!kubectl apply -f k8s/hpa.yaml
!kubectl get hpa -n my-app

## Проверка и тестирование

In [ ]:
# Получение URL Istio Ingress Gateway
!kubectl get svc istio-ingressgateway -n istio-system

# Тестирование приложения
import requests

GATEWAY_URL = "http://a1234567890.us-east-1.elb.amazonaws.com"

# Health check
response = requests.get(f"{GATEWAY_URL}/api/health")
print(f"Health Check: {response.status_code}")

# Load test
!kubectl run -i --tty load-generator --rm --image=busybox --restart=Never -- /bin/sh -c "while sleep 0.01; do wget -q -O- http://api-gateway:8000/api/health; done"

## Заключение

### Развернутая инфраструктура:

✅ EKS кластер с node groups (on-demand + spot)  
✅ Istio Service Mesh для управления трафиком  
✅ ALB Ingress Controller  
✅ ArgoCD для GitOps  
✅ Prometheus + Grafana для мониторинга  
✅ HPA для автоскейлинга подов  
✅ Cluster Autoscaler для автоскейлинга нод  

### Полезные команды:

```bash
# Просмотр логов
kubectl logs -f deployment/api-gateway -n my-app

# Istio dashboard
istioctl dashboard kiali

# Grafana dashboard
kubectl port-forward -n monitoring svc/prometheus-grafana 3000:80

# ArgoCD UI
kubectl port-forward svc/argocd-server -n argocd 8080:443
```